In [54]:
# initial imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import panel as pn
import numpy as np
import datetime as dt
from dotenv import get_key
from pathlib import Path
pn.extension('plotly')
import warnings
warnings.filterwarnings('ignore')

In [55]:
import hvplot.pandas
import matplotlib
#%matplotlib inline
import psycopg2
from sqlalchemy import create_engine
import json

In [56]:
#backtesting library
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
backtesting.set_bokeh_output(notebook=False)
import alpaca_trade_api as tradeapi

In [57]:
global stock
global slow
global fast
global cash
global order
global commission

In [58]:
def run_backtest(event):
    run_bt_button.name =slow_sma.value
    panel_tabs_view.active=3
    #remove_obj()
    slow = int(slow_sma.value)
    fast =  int(fast_sma.value)
    cash= int(cash_val.value)
    stock= ticker_stk.value
    commission=float(commission_pct.value)
    order = order_type.value
    file = './resources/' + stock +'.csv'
    stock=pd.read_csv(file, index_col='Date', parse_dates=True)
    bt = Backtest(stock, SmaCross, cash=cash, commission=commission, exclusive_orders=order_type)                                                     
    output = bt.run()
    plot= bt.plot(open_browser=True, plot_width=1500, plot_equity=True, plot_pl=True, plot_volume=True,
    plot_drawdown=True, smooth_equity=False, relative_equity=True, resample=True, reverse_indicators=False, show_legend=True)
    bt_config.append(plot)
    bt_config.append(output)
    panel_tabs_view[3] = ("Backtest & Benchmark", bt_config)
    panel_tabs_view.active=3
    #return bt_config

In [59]:
class SmaCross(Strategy):
    sma_slow = 100
    sma_fast= 10
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.sma_fast)
        self.sma2 = self.I(SMA, self.data.Close, self.sma_slow)
        
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


In [60]:
def initializer(event):
    """Function to initiate interactive widgets and build the panel based on user selection.
    this is tied to start button onclick event which loads additional widgets for selecting data sources and asssociate 
    parameters such as the files, api keys etc."""
    if start_button.name =='Start':
        panel_tabs_view.active = 0
        inital_right.append(config_col)
        start_button.name='Cancel'
        if wizard_group.value=='Configure Strategy':
            #config_bt_strategy()       
            #bt_config.clear()
            panel_tabs_view[3] = ("Backtest & Benchmark", bt_pane)
            panel_tabs_view.active=3 
            
    elif start_button.name == 'Cancel':
        inital_right.remove(config_col)
        start_button.name='Start'

In [61]:
def load_file_data(event):
    """function identifies file extensions and triggers dataload by calling get_Dataframe function"""
    data_col.remove
    panel_tabs_view.active=0
    files_to_load = select_data_files.value
    df_dict={}
    for file in files_to_load:
        file_extention = file.split(".")[1]
        file_desc = file.split(".")[0]
        file_desc = "r"+f'{file_desc}'
        file_name = file_desc.split('\\')[-1]
        abs_path = file.replace('\\', '/')
        df_dict.update(get_dataframe(file_extention, file_name, abs_path))
    paint_panel_tab(df_dict)

In [62]:
def paint_panel_tab(df_dict):
    """function to populate the data screen/panel"""
    data_row.clear()
    data_col.clear()
    row_index=0
    for key in df_dict:
        df_pane = pn.pane.DataFrame(df_dict[key],  render_links = True, show_dimensions =True, max_rows =5)
        pane = pn.panel(df_pane.param, parameters=['index', 'show_dimensions', 'max_rows',],
        widgets={'max_rows': {'start': 1, 'end': len(df_dict[key]), 'value': 5}})
        #df_plot_graph = get_financial_plots(df_dict[key])
        null_count = pn.widgets.StaticText(name='Nulls: ', value=df_dict[key].count() - df_dict[key].notna().count())
        data_types = pn.widgets.StaticText(name='Data types: ', value=df_dict[key].dtypes)
        duplicate_data = pn.widgets.StaticText(name='Duplicates: ', value = df_dict[key].count() - df_dict[key].nunique())
        data_col.insert(row_index, pn.Column())
        data_col[row_index].append(key)
        data_col[row_index].append(df_pane)
        data_col[row_index].append(null_count)
        data_col[row_index].append(data_types)
        data_col[row_index].append(duplicate_data)
        data_col[row_index].append(pane)
        data_col[row_index].append(visual_row)
        plots = plot_graphs(df_dict[key])
        for plot in plots:
            data_col[row_index].append(plot)
        data_col[row_index].append(reset_button)
        data_row.insert(row_index, pn.Row(data_col[row_index]))
        row_index += 1
    panel_tabs_view.active = 1

In [63]:
def show_plot(event):
    try:
        plot_btn.width = 50
        plot_btn.background = 'Red'
        plot_btn.disabled=True    
        for plot in plots:
            data_col.append(plot)
            panel_tabs_view[2] = ("Visualizations", data_col)
            panel_tabs_view.active = 2
    except Exception as e:
        plot_btn.message(e)

In [64]:
def plot_selected(event): # not used
    selected_plot = plot_option.value
    if selected_plot != "Select Chart":
        selected_plot = selected_plot.lower()
        selected_plot =  globals()[selected_plot]
        #show_btn.name=plot_option.value
        selected_plot()
        #return selected_plot()#(stock)

In [65]:
def plot_graphs(df_item):
    """function to plot graphs.  This is base version, ideally a variable specifying 
    the type of graph needed will need to be passed upon user selection"""
    df_item.dropna(inplace = True)
    df_item= df_item.sort_index()
    dr_plot= df_item.hvplot.line( title = " Daily Close Vs SMA20, 50 & 200",  grid=True)
    sma5 = np.round(df_item.rolling(window = 5).mean(), 2)
    #sma5_plt = sma5.hvplot.line(x='Date', y='Close', grid=True, title="5 day Moving average")
    sma20 = np.round(df_item.rolling(window = 20).mean(), 2)
    sma20_plot = sma20.hvplot.line( grid=True, title="20 day Moving average")
    sma50 = np.round(df_item.rolling(window = 50).mean(), 2)
    sma50_plot = sma50.hvplot.line(  grid=True, title="50 day Moving average")
    sma200 = np.round(df_item.rolling(window = 200, center = False).mean(), 2)
    sma200_plot = sma200.hvplot.line( grid=True, title="200 day Moving average")
    sma_combo_plot =  dr_plot* sma20_plot*sma50_plot*sma200_plot
    sma_combo_plot.opts(bgcolor='black')
    if 'high' in df_item.columns:
        candle = go.Figure(data=[go.Candlestick(x=df_item.index,
                        open = df_item['open'],
                        high =df_item['high'],
                        low=df_item['low'],
                        close=df_item['close'])])
    else:
        candle=df_item['close'].hvplot.line()
     
    daily_returns = df_item['close'].pct_change().hvplot.line(title="Daily, Cumulative 180 day STD & Mean")
    cumulative_returns = (1 + df_item['close'].pct_change()).cumprod().hvplot.line()
    std_180=df_item['close'].pct_change().rolling(window=180).std().hvplot.line()
    mean_180= df_item['close'].pct_change().rolling(window=180).mean().hvplot.line()
    ret_combo_plot =  daily_returns * cumulative_returns * std_180 * mean_180
    ret_combo_plot.opts(bgcolor='black')
    return [candle, sma_combo_plot, ret_combo_plot]

In [66]:
def get_dataframe (ext, name, path):
    """This is where we read each file and load the data into dataframes"""
    df_dict = {}
    if ext == 'xlsx':
        data_frame = pd.read_excel(path, parse_dates=True, infer_datetime_format=True)
    elif ext == 'csv':
        data_frame =pd.read_csv(path, index_col='Date', parse_dates=True)
    elif ext == 'json':
        data_frame = pd.read_json(path, orient=None, typ='frame')
    elif ext == 'sql':
        data_frame == pd.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None)
    elif ext == 'html':
        data_frame == pd.read_html(io, match='.+', flavor=None, header=None, index_col=None, skiprows=None, attrs=None, parse_dates=False, tupleize_cols=None, thousands=',', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
    else:
        print("cannot read the file extensions")
    df_dict= {name: data_frame}
    return df_dict

In [67]:
def fetch_api_data(event):
    """This is where we fetch data using API and pass the data frame to the plotting function"""
    df_dict={}
    alpaca_key=api_key.value
    secret_key=api_secret_key.value
    alpaca = tradeapi.REST(alpaca_key, secret_key, api_version="v2")
    today=pd.Timestamp("2020-08-19", tz="America/Chicago").isoformat()
    timeframe="1D"
    tf_start = start_date.value
    tf_end= end_date.value
    tickers = ticker_selector.value
    for ticker in tickers:
        df_portfolio = alpaca.get_barset(ticker, timeframe, start=tf_start, end=tf_end).df
        df_dict.update({ticker: df_portfolio[ticker]})
    paint_panel_tab(df_dict)

In [68]:
#fetch_api_data()

In [69]:
def sector_analysis():
    # use the Data frame created by Luis
    """function to plot the sector wise charts like in https://www.sectorspdr.com/sectorspdr/tools/sector-tracker"""
    pass 

def data_cleanser():
    """function to inspect, clean, manipulate the data"""
    pass

def simulate_mc():
    """place holder function to initiate mc simulation"""
    pass

In [70]:
def clearpane(event):
    """ideally pass the row/column reference. this is a place holder function"""
    #data_row.clear()
    data_col.clear()
    #remove_obj()
    panel_tabs_view.active=0

In [71]:
def show_source(event):
    """function to show/ remove widget for loading data sources"""
    if data_sources.value == 'API':
        remove_obj()
        config_col.append(api_row)
        config_col.append(date_row)
        config_col.append(ticker_selector)
        config_col.append(reset_button)
        config_col.append(fetch_button)
    elif data_sources.value == 'Local Files':
        remove_obj()
        config_col.append(select_data_files)
        config_col.append(load_files_button)
    elif data_sources.value == 'Google Finance':
        remove_obj()
        config_col.append(url_input)
    elif data_sources.value == 'Database':
        remove_obj()
        config_col.append(db_source)
        config_col.append(db_user)
        config_col.append(db_pswd)
        config_col.append(db_conn)
        config_col.append(db_sql)
        config_col.append(dbconn_button)
        config_col.append(reset_button)
    else:
        remove_obj()

In [72]:
def db_connect(event):
    engine = db_source.value + f'://' + db_user.value  + f':' + db_pswd.value + "@" + db_conn.value + "/" + db_user.value
    pgsql_engine = create_engine(engine)
    db_connection = pgsql_engine.connect()
    df_sql = pd.read_sql(db_sql.value, db_connection)
    df_pane = pn.pane.DataFrame(df_sql,  render_links = True, height_policy = 'auto', width_policy = 'auto', align='start', sizing_mode='scale_both', show_dimensions =True, max_rows =5)
    panel_tabs_view[2] = ("Visualizations", df_pane)
    panel_tabs_view.active=2
    config_col.clear()
    panel_tabs_view[2].append(config_col)
    config_col.append(df_sql)
    config_col.append(query_builder)
    plot_lst=[]
    for plot in px:
        plot_lst.append(plot)
    plot_lst = pn.widgets.Select(name='Visualizations', options=plot_lst)
    config_col.append(plot_lst)

In [73]:
def remove_obj():
    """function to clear previous widgets"""
    for widget in config_col.objects:
        if widget.name != 'DataSource':
            config_col.remove(widget)
            pass

In [74]:
#test function calls
#load_file_data() 
# show_plot()

In [75]:
#clearpane()

In [76]:
#run_backtest()

In [77]:
#paint_panel_tab(df_dict)

In [78]:
welcome_img = pn.pane.PNG('Resources/images/financial-planner.png', width=300)

In [79]:
header= """#Portfolio Builder and Algorithmic Trading Platform"""

markdown ="""This tool helps build, compare, backtest, benchmark and simulate porfolio returns 
based on style factors, sector or different Indexes with choice to either run 
in the background or in an interactive way providing a wizard at every step for the advisor 
to analyze the visualizations and calibrate Trading strategies."""

welcome_msg = pn.Column(header, welcome_img, markdown )

In [80]:
start_date = pn.widgets.DatePicker(name='StartDate')
end_date = pn.widgets.DatePicker(name='EndDate')
date_row = pn.Row(start_date, end_date)

In [81]:
url_input = pn.widgets.TextInput(name='google_link', value='Enter the google finance url')
db_source = pn.widgets.TextInput(name='Database', value='Enter Database name')
db_conn = pn.widgets.TextInput(name='Connection', value='localhost:5432')
db_user = pn.widgets.TextInput(name='usrname', value='enter DB user name')
db_pswd = pn.widgets.PasswordInput(name='DBPassword', max_width = 200, placeholder='Enter Database password')
db_sql = pn.widgets.TextInput(name='Sql', value='enter the SQL query')

In [82]:
ticker_df = pd.read_csv('./resources/sp500_companies.csv', header='infer', delimiter=',', index_col='Symbol', usecols=['Symbol']).dropna()
options=[]
for ticker in ticker_df.index:
    options.append(ticker)
ticker_selector = pn.widgets.CrossSelector(name='Tickers', disabled=False,
    options=options)
ticker_selector

CrossSelector(name='Tickers', options=['MMM', 'AOS', ...])

In [83]:
ticker_selector.value

[]

In [84]:
select_data_files = pn.widgets.FileSelector('~')
#select_data_files

In [85]:
selected_files = select_data_files.value
#selected_files

In [86]:
file_input = pn.widgets.FileInput(accept='.csv,.json, .xlsx')
#file_input

In [87]:
visualization_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Tearsheet', 'close', 'beta', 'cor', 'SMA', 'EWMA'], options=['Full Tearsheet', 'Daily close', 'beta', 'Correlation', 'SMA', 'EMA'],
    inline=True)
create_visual = pn.widgets.Button(name='Create', button_type='primary', width=20)
visual_row = pn.Column(visualization_group, create_visual)
visual_row

Column
    [0] CheckBoxGroup(inline=True, name='Checkbox Group', options=['Full Tearsheet', ...], value=['beta', 'SMA'])
    [1] Button(button_type='primary', name='Create', width=20)

In [88]:
run_mc_button = pn.widgets.Button(name='Run MC Simulation', button_type='primary')
load_files_button = pn.widgets.Button(name='Load Data files', button_type='primary')
reset_button = pn.widgets.Button(name='Reset', button_type='primary')
start_button = pn.widgets.Button(name='Start', button_type='primary')
fetch_button = pn.widgets.Button(name='Fetch Data', button_type='primary')
dbconn_button = pn.widgets.Button(name='Query Database', button_type='primary')
query_builder = pn.widgets.Button(name='generate Query', button_type='primary')
plot_btn = pn.widgets.Button(name='🔍', button_type='primary', width=5)
plot_btn.on_click(show_plot)
reset_button.on_click(clearpane)
run_mc_button.on_click(simulate_mc)
dbconn_button.on_click(db_connect)

In [89]:
wizard_group = pn.widgets.RadioBoxGroup(name='RadioBoxGroup', 
               options=['Step by Step Wizard', 'Visualizations Wizard', 'Configure Strategy', 'Just show me the Money'])

wizard_group

RadioBoxGroup(options=['Step by Step Wizard', ...], value='Step by Step Wizard')

In [90]:
#defining wigets for collecting user input and associating them to a column. 
data_sources = pn.widgets.Select(name='DataSource', max_width = 200, options=['Select a data source', 'Local Files', 'Google Finance', 'API', 'Database'])
data_sources.param.watch(show_source, 'value')
api_sources = pn.widgets.Select(name='APISource', options=['Quandl', 'Alpaca', 'Quantopia', 'Postman', 'other'])
api_key = pn.widgets.PasswordInput(name='API Key', max_width = 200, placeholder='Enter API key/Token here...')
api_secret_key = pn.widgets.PasswordInput(name='Secret Key', max_width = 200, placeholder='Enter secret key/Token here...')
api_row = pn.Row(api_sources, api_key, api_secret_key)
config_col = pn.Column(data_sources, width_policy='auto')
inital_right = pn.Column("How would you like to proceed?", wizard_group, start_button)
inital_left= pn.Column(welcome_msg)
Initialization_pane = pn.Row(inital_left, inital_right, background='#f0f0f0')
bt_config = pn.Row("Configure BackTest Strategy", reset_button)
macd_pane=pn.panel(bt_config)
config_col
data_col = pn.Column(reset_button)
data_row = pn.Row("verify Data")
data_pane = pn.panel(data_row)
start_button.on_click(initializer)
load_files_button.on_click(load_file_data)
fetch_button.on_click(fetch_api_data)

In [91]:
panel_tabs_view = pn.Tabs(
     ("Welcome", Initialization_pane),
     ("Base Data", data_pane),
     ("Compare Portfolio"),
     ("Backtest & Benchmark", macd_pane),
     ("Simulate"), dynamic=True,
        height_policy = 'auto', width_policy = 'auto', align='start', sizing_mode='scale_both', tabs_location='right')

In [92]:
#def config_bt_strategy():
    #panel_tabs_view[3].clear()
config_bt=pn.widgets
slow_sma = pn.widgets.TextInput(name='slow')#, value='200')
fast_sma = pn.widgets.TextInput(name='fast')#, value='20')
ticker_stk = pn.widgets.Select(name='stock', max_width = 100, options=['AMD', 'GOOG'])
commission_pct = pn.widgets.TextInput(name='commission', value='0.002')
cash_val = pn.widgets.TextInput(name='cash', value='100000')
order_type = pn.widgets.Select(name='Order', max_width = 100, options=['True', 'False'])
#config=[slow_sma, fast_sma, ticker_stk, commission_pct, cash_val, order_type]
run_bt_button = pn.widgets.Button(name='RunBacktest', button_type='primary')
run_bt_button.on_click(run_backtest)   
config_bt_col = pn.Column('Configure Backtest Strategy')
config_sma_row = pn.Row(ticker_stk, slow_sma, fast_sma)
config_order_row = pn.Row(cash_val, commission_pct, order_type)                            
config_bt_col.append(config_sma_row)
config_bt_col.append(config_order_row)
config_bt_col.append(run_bt_button)
bt_pane=pn.Pane(config_bt_col)
panel_tabs_view[3] = ('Configure Backtest Strategy', bt_pane)
#panel_tabs_view.active=3 
#return view         

In [93]:
# global selected_plot
# viz_options=['Select Chart']

# for plot in plot_selector:
#     if plot[0] != '_' and 'plot' not in plot :
#         viz_options.append(plot)

# plot_option =  pn.widgets.Select(name='Select Visualization', options=viz_options)
# show_btn = pn.widgets.Button(name='Plot', button_type='primary', width=1)
# # selected_plot=plot_option.value

# show_btn.on_click(plot_graph)
# btn_row= pn.Row(plot_option, show_btn) 
# btn_row

# plot_option = pn.interact(plot_graph, chart=viz_options)
# panel_plot = pn.panel(plot_option)
# panel_plot

In [94]:
panel_tabs_view.show()

Launching server at http://localhost:56357


In [95]:
def get_financial_plots(): # test function
    "function to create financial ratio plots"
    df_item = df_item.sort_index()
    if 'High' in df_item.columns:
        candle = go.Figure(data=[go.Candlestick(x=df_item.index,
                        open = df_item['Open'],
                        high =df_item['High'],
                        low=df_item['Low'],
                        close=df_item['Close'])])
    else:
        candle=df_item['Close'].hvplot.line()
     
    daily_returns = df_item['Close'].pct_change().hvplot.line()
    cumulative_returns = (1 + df_item['Close'].pct_change()).cumprod().hvplot.line()
    std_180=df_item['Close'].rolling(window=180).std().hvplot.line()
    mean_180= df_item['Close'].rolling(window=180).mean().hvplot.line()
    sma50 = np.round(df_item.rolling(window = 50).mean(), 2)
    sma50_plot = sma50.hvplot.line(x='Date', y='Close',  grid=True, title="50 day Moving average")
    combo_plot =  daily_returns * cumulative_returns * std_180 * mean_180 * sma50_plot
    #return combo_plot
#amd_df =pd.read_csv('./resources/amd.csv', index_col='Date', parse_dates=True)



In [96]:
#get_financial_plots(amd_df)
